In [1]:
#import
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit import Aer, execute
from qiskit.circuit.library import MCXGate
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile

In [2]:
#CORRECT OFR IMPLEMENTING NEAGTIVE NUMBER 
import numpy as np
def to_twoscomplement(bits, value):
    if value < 0:
        value = ( 1<<bits ) + value
    formatstring = '{:0%ib}' % bits
    return formatstring.format(value)

In [3]:
# qram for storing numbers with indices
def forward_qram_n(qc, addr , mem, n,m, input_vector): 
    for (i, x) in enumerate(input_vector):
        add_x = []
        for (j, bit) in enumerate(to_twoscomplement(n, i)):
            if bit == '0': add_x.append(addr[j])
        if add_x: qc.x(add_x)
        target = []
        for (j, bit) in enumerate(to_twoscomplement(m, x)):
            if bit == '1': target.append(mem[j])
        for register in target: 
            qc.mcx(addr, register)
        if add_x: qc.x(add_x)
        qc.barrier(mem, addr)

In [4]:
# oracle for taget state 
def oracle_n(qc, clause, mem, m, output):
    clause_binary = to_twoscomplement(m, clause)[::-1]
    qc.append(MCXGate(m, ctrl_state=clause_binary), mem[0:m] + [output]) 

In [5]:
# diffuser 
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    return qc

In [6]:
# Input List
input_vector = [-15,-14,2,-1]

In [7]:
# Determing taget state
for num in input_vector:
    if num <= 0:
        negative_number = num

clause = negative_number

In [8]:
# number of qbits for indices
n = int(np.ceil(np.log2(len(input_vector))))
# number of qbits for large negative numbers
m = 8
addr = QuantumRegister(n,'addr')
mem = QuantumRegister(m, 'mem')
c = ClassicalRegister(n)
qc = QuantumCircuit(addr,mem,c)
output_qubit = QuantumRegister(1, name='out')
qc = QuantumCircuit(addr, mem, output_qubit, c)
qc.initialize([1, -1]/np.sqrt(2), output_qubit)    
qc.h(addr)

qc.barrier() 
forward_qram_n(qc, addr, mem, n,m, input_vector)

oracle_n(qc, clause, mem, m, output_qubit)
qc.barrier() 
forward_qram_n(qc, addr, mem, n,m, input_vector)

qc.append(diffuser(n), addr)
forward_qram_n(qc, addr, mem, n,m, input_vector)

oracle_n(qc, clause, mem, m, output_qubit)
qc.barrier() 
forward_qram_n(qc, addr, mem, n,m, input_vector)

qc.append(diffuser(n), addr)
forward_qram_n(qc, addr, mem, n,m, input_vector)

oracle_n(qc, clause, mem, m, output_qubit)
qc.barrier() 
forward_qram_n(qc, addr, mem, n,m, input_vector)

qc.append(diffuser(n), addr)
forward_qram_n(qc, addr, mem, n,m, input_vector)

oracle_n(qc, clause, mem, m, output_qubit)
qc.barrier() 
forward_qram_n(qc, addr, mem, n,m, input_vector)

qc.append(diffuser(n), addr)

qc.measure(addr, c[0:n])
#qc.measure(mem, c[n:n])


qc = qc.reverse_bits()

aer_sim = Aer.get_backend('aer_simulator')
compiled_circuit = transpile(qc, aer_sim)
result = aer_sim.run(compiled_circuit).result()
counts = result.get_counts()
indice= int(max(counts, key=counts.get), 2)
candidate = input_vector[indice]
if clause == candidate:
    print(True)

True
